In [584]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [585]:
fname = "test.tsv"
dataset = pd.read_csv(fname, sep="\t")
X = dataset.iloc[:, :].values

In [586]:
X = np.array([
  [
    "SendClearCard > ID",
    "AN14",
    "N/A",
    "O",
    "hello world 5"
  ],
  [
    "SendClearCard > Statements > Stat > SSS",
    "AN14",
    "N/A",
    "O",
    "hello world 4"
  ],
  [
    "Acquirer > a > a > a > a > a",
    "AN14",
    "N/A",
    "O",
    "hello world 5"
  ],
  [
    "Acquirer > b > b > b > b > b > b > b > b",
    "AN14",
    "N/A",
    "O",
    "hello world 5"
  ],
  [
    "Acquirer > c",
    "AN14",
    "N/A",
    "O",
    "hello world 5"
  ],
  [
    "cardAccpetor > dd",
    "AN14",
    "N/A",
    "O",
    "hello world 5"
  ],
]
)

In [587]:
X

array([['SendClearCard > ID', 'AN14', 'N/A', 'O', 'hello world 5'],
       ['SendClearCard > Statements > Stat > SSS', 'AN14', 'N/A', 'O',
        'hello world 4'],
       ['Acquirer > a > a > a > a > a', 'AN14', 'N/A', 'O',
        'hello world 5'],
       ['Acquirer > b > b > b > b > b > b > b > b', 'AN14', 'N/A', 'O',
        'hello world 5'],
       ['Acquirer > c', 'AN14', 'N/A', 'O', 'hello world 5'],
       ['cardAccpetor > dd', 'AN14', 'N/A', 'O', 'hello world 5']],
      dtype='<U40')

In [588]:
count = len(X[0, :])
if count == 5:
    isRequst = True
else:
    isRequst = False

In [589]:
mobj = fname[0:fname.index(".")]
main_name = mobj

In [590]:
c = mobj[0].capitalize()
mobj = c + mobj[1:]
mobj
if isRequst:
    mobj += "Type"
else:
    mobj += "ResponseType"

In [591]:
mobj

'TestType'

In [592]:
result = f"""
    {mobj}:
      type: object
      properties:
"""

In [593]:
print(result)


    TestType:
      type: object
      properties:



In [594]:
result += f"""        {main_name}:
          type: object
          properties: 
"""

In [595]:
print(result)


    TestType:
      type: object
      properties:
        test:
          type: object
          properties: 



In [596]:
def camel_case(string):
  c = string[0].lower()
  string = c + string[1:]
  return string

In [597]:
def spaces(string):
  return ''.join(['  ' for s in range(string.count('>'))])

In [598]:
def extractNandM(string:str):
  n = ""
  m = ""
  nfound = True
  ffound = False
  for el in string:
    if el.isdigit():
      ffound = True
      if nfound:
        n += el
      else:
        m += el
    else:
      if ffound:
        nfound = False
  return [n, m]

In [599]:
def makeTSpaces(num, mCount=0, mmCount=0):
  if mmCount > 1 and mCount != 0:
    num += mmCount 
  return ''.join(['  ']*num)

In [600]:
def checkDefault(default):
  if type(default) != float:
    return f"""default: {default}"""
  else:
    return ""

In [601]:
# from cmath import nan
# from dataDictionary import makeDataDic

# def makeObject(lst, obj):
#   tag = lst[0]
#   data = lst[1].replace(" ", "")
#   default = lst[2]
#   req = lst[3].strip(' ')
#   des = lst[-1]
#   n, m = extractNandM(data)
#   dataDic = makeDataDic(n,m, makeTSpaces(7))
#   obj = """"""
#   if tag.count('>') == 0:
#     obj += f"""{makeTSpaces(6)}{camel_case(tag)}:
# {makeTSpaces(7)}{dataDic[data]}
# {makeTSpaces(7)}{checkDefault(default)}
# {makeTSpaces(7)}description: >
# {makeTSpaces(8)}{des}"""
#     return obj
#     if tag.count('>') > 0:
#       pass

In [602]:
# l = ["request", "N10", "N/A", "hello everyone"]
# o = makeObject(l, acquirerObj)
# print(o)

In [603]:
# n = len(X)
# flagA = True
# flagC = True
# flagCA = True
# acquirerObj = f"""    Acquirer_{main_name.capitalize()}:
#       type: object
#       properties:
# """
# for i in range(n):
#   first = camel_case(required(tag[i])[0])
#   acquirerObj += makeObject(X[i], acquirerObj)
#   if first == "acquirer": 
#     if flagA:
#       result += f"""            acquirer:
#               $ref: '#/components/schemas/Acquirer_{main_name.capitalize()}'
# """
#       flagA = False

In [604]:
def ismatchany(pre, curr):
    i = 0
    mCount = 0
    tCount = len(pre) if len(pre) > len(curr) else len(curr)
    while i < len(pre) and i < len(curr):
      if pre[i] == curr[i]:
        mCount += 1
      else:
        break
      i += 1
    return [True if mCount > 0 else False, mCount, tCount - mCount]

In [605]:
from re import sub
def camel_case(s):
  return ''.join([s[0].lower(), s[1:]])

In [606]:
import os
from dataDictionary import makeDataDic

def makeObject(lst, pre, ts=0, o=0, dSpaces=2):
  obj = """"""
  tag = lst[0]
  data = lst[1].replace(' ', '')
  if isRequst:
    default = lst[2]
    req = lst[3]
  else:
    default = None
    req = None
  des = lst[-1]
  tag = tag.split('>')
  tag = [el.replace(' ', '') for el in tag]
  tag = [camel_case(el) for el in tag]
  tLength = len(tag)

  n, m = extractNandM(data)

  ismatch, mCount, mmCount = ismatchany(pre, tag)
  
  if ismatch:
    tag = tag[mCount:]
    if len(pre) >= tLength:
      o -= (mmCount - 1)
      ts -= mmCount
    else:
      o = mCount
      ts = mCount
  else:
    o = 0
    ts = 0
  
  length = len(tag)

  i = 0
  while i < length-1:
    obj += f"""{makeTSpaces(o+ts+dSpaces)}{tag[i].lower() if len(tag[i]) < 4 else tag[i]}:\n"""
    ts += 1
    obj += f"""{makeTSpaces(o+ts+dSpaces)}type: object\n"""
    obj += f"""{makeTSpaces(o+ts+dSpaces)}properties:\n"""
    o += 1
    i += 1

  obj += f"""{makeTSpaces(o+ts+dSpaces)}{tag[i].lower() if len(tag[i]) < 4 else tag[i]}:\n"""
  ts += 1
  obj += f"""{makeTSpaces(o+ts+dSpaces)}{makeDataDic(n, m, makeTSpaces(o+ts+dSpaces))[data]}\n"""
  default = checkDefault(default)
  if default:
    obj += f"""{makeTSpaces(o+ts+dSpaces)}{default}\n"""
  obj += f"""{makeTSpaces(o+ts+dSpaces)}description: >
{makeTSpaces(o+ts+1+dSpaces)}{des}\n"""
  return obj, ts, o

pre = []
if isRequst:
  dSpaces = 2
else:
  dSpaces = 4
bigObj = """"""
ts = 0
o = 0
for row in X:
  curr = row
  obj, ts, o = makeObject(curr, pre, ts, o, dSpaces)
  bigObj += obj
  pre = curr[0].replace(" ", "").split('>')
  pre = [camel_case(el) for el in pre]
bigObj = os.linesep.join([s for s in bigObj.splitlines() if s])
print(bigObj)

    sendClearCard:
      type: object
      properties:
        id:
          type: string
          format: AN14
          default: N/A
          description: >
            hello world 5
        statements:
          type: object
          properties:
            stat:
              type: object
              properties:
                sss:
                  type: string
                  format: AN14
                  default: N/A
                  description: >
                    hello world 4
    acquirer:
      type: object
      properties:
        a:
          type: object
          properties:
            a:
              type: object
              properties:
                a:
                  type: object
                  properties:
                    a:
                      type: object
                      properties:
                        a:
                          type: string
                          format: AN14
                          default: N/A
    